In [128]:
# 데이터 불러오기
import pandas as pd
import numpy as np 
import os

data_origin = pd.read_csv("데이터 전처리_혜원.CSV", header = 1)


In [129]:
# 쓸만한 칼럼 보기 위해서 우선 전체 칼럼 리스트 출력
data_origin.columns

Index(['매장명', 'date', 'day', 'season', 'temperature', 'weather', 'cloudy',
       'rain', 'snow', 'sunny', 'airpolution', 'humid(%)', '코로나 전 후', '품목 분류',
       'labeling', 'menu', '총수량', '총 금액', '순이익', '08시 수량', '08시 금액', '09시 수량',
       '09시 금액', '10시 수량', '10시 금액', '11시 수량', '11시 금액', '12시 수량', '12시 금액',
       '13시 수량', '13시 금액', '14시 수량', '14시 금액', '15시 수량', '15시 금액', '16시 수량',
       '16시 금액', '17시 수량', '17시 금액', '18시 수량', '18시 금액', '19시 수량', '19시 금액',
       '20시 수량', '20시 금액'],
      dtype='object')

In [130]:
# 날씨, 요일, 메뉴를 보려고 함
import seaborn as sns
import matplotlib.pyplot as plt

data_weather_menu = data_origin[['day', 'season', 'temperature', 'cloudy',
       'rain', 'snow', 'sunny', 'airpolution', 'humid(%)',
       'labeling' , 'menu']]

# sns.set(font="Malgun Gothic", 
#         rc={"axes.unicode_minus":False},
#         style='darkgrid')
# sns.pairplot(data_origin)

# sns.pairplot(data_origin[col])
# plt.show()

In [131]:
# 칼럼 삭제 전 shape보기 (행,열)
print(data_weather_menu.shape)

(21599, 11)


In [132]:
# 상위 5개 품목은 어떤 요인과 관련없이 다 잘나가기 때문에 해당 음료는 제외하고 보기
# 해당 음료가 있는 행의 인덱스 반환 후 인덱스 기준으로 삭제하기
drop_index = data_weather_menu[(data_weather_menu["menu"] == '아이스 아메리카노')|(data_weather_menu["menu"] == '아이스 카페라떼')|(data_weather_menu["menu"] == '아이스 바닐라라떼')|(data_weather_menu["menu"] == '딸기라떼')|(data_weather_menu["menu"] == '아메리카노')].index
#아이스 아메리카노, 아이스 카페라떼, 아이스 바닐라라떼, 딸기라떼, 아메리카노

print(drop_index)

Int64Index([    1,     2,     3,     9,    36,    38,    39,    41,    51,
               67,
            ...
            21552, 21553, 21554, 21556, 21565, 21577, 21579, 21580, 21583,
            21590],
           dtype='int64', length=2563)


In [133]:
# inplace = True : 변경을 저장하겠다. 
data_weather_menu.drop(drop_index, inplace=True)

C:\Users\hop09\AppData\Local\Temp\ipykernel_9952\352717995.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_weather_menu.drop(drop_index, inplace=True)


In [134]:
# 기존보다 행 개수가 준 것을 확인할 수 있음 
data_weather_menu.shape

(19036, 11)

In [135]:
# 가지고 있는 칼럼 보기 -> 더미변수화를 위해
data_weather_menu.columns

Index(['day', 'season', 'temperature', 'cloudy', 'rain', 'snow', 'sunny',
       'airpolution', 'humid(%)', 'labeling', 'menu'],
      dtype='object')

In [136]:
# 메뉴, 계절, 요일은 더미변수화해서 보는 것이 좋을 것 같다고 판단, 더미변수화 진행
data_dummies_menu = pd.DataFrame(pd.get_dummies(data_weather_menu['menu']))
data_dummies_season = pd.DataFrame(pd.get_dummies(data_weather_menu['season']))
data_dummies_day = pd.DataFrame(pd.get_dummies(data_weather_menu['day']))

In [137]:
# 새로운 데이터프레임 만들어주기
data_target = data_weather_menu[['labeling']]
data_weather_menu = pd.concat([data_weather_menu[['cloudy',
       'rain', 'snow', 'sunny', 'airpolution', 'humid(%)', 'temperature',
      ]] ,data_dummies_season, data_dummies_day, data_dummies_menu, data_target], axis = 1)

In [138]:
print(data_weather_menu.shape)

(19036, 189)


In [ ]:
# heatmap그리는 코드
# import seaborn as sns
# import matplotlib.pyplot as plt


# sns.set(font="Malgun Gothic", 
#         rc={"axes.unicode_minus":False},
#         style='darkgrid')
# fig, ax = plt.subplots(1,1 ,figsize = (40,40))
# cmap = sns.light_palette("darkgray", as_cmap=True)
# sns.heatmap(data_weather_menu.corr(), annot=True, cmap=cmap)
# plt.show()

In [108]:
# 상관분석 진행
pd.set_option('display.max_rows', None)
data_weather_corr = pd.DataFrame(data_weather_menu.corr(method='pearson'))
data_weather_corr
data_weather_corr.to_excel("날씨품명cor분석.xlsx", encoding='cp949')

C:\Users\hop09\AppData\Local\Temp\ipykernel_9952\964632985.py:3: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  data_weather_corr = pd.DataFrame(data_weather_menu.corr(method='pearson'))
c:\Users\hop09\Desktop\anaconda\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


## 코로나 전 cor분석

In [115]:
# 데이터 불러오기
import pandas as pd
import numpy as np 
import os

data_origin = pd.read_csv("데이터전처리_혜원_코로나전.CSV", header = 0)




In [117]:
# 날씨, 요일, 메뉴를 보려고 함
import seaborn as sns
import matplotlib.pyplot as plt

data_weather_menu = data_origin[['day', 'season', 'temperature', 'cloudy',
       'rain', 'snow', 'sunny', 'airpolution', 'humid(%)',
       'labeling' , 'menu']]

In [118]:
print(data_weather_menu.shape)
# 상위 5개 품목은 어떤 요인과 관련없이 다 잘나가기 때문에 해당 음료는 제외하고 보기
# 해당 음료가 있는 행의 인덱스 반환 후 인덱스 기준으로 삭제하기
drop_index = data_weather_menu[(data_weather_menu["menu"] == '아이스 아메리카노')|(data_weather_menu["menu"] == '아이스 카페라떼')|(data_weather_menu["menu"] == '아이스 바닐라라떼')|(data_weather_menu["menu"] == '딸기라떼')|(data_weather_menu["menu"] == '아메리카노')].index
#아이스 아메리카노, 아이스 카페라떼, 아이스 바닐라라떼, 딸기라떼, 아메리카노

print(drop_index)
# inplace = True : 변경을 저장하겠다. 

(16697, 11)
Int64Index([    1,     2,     3,     9,    36,    38,    39,    41,    51,
               67,
            ...
            16555, 16557, 16558, 16568, 16584, 16632, 16633, 16634, 16642,
            16659],
           dtype='int64', length=1390)


In [119]:
data_weather_menu.drop(drop_index, inplace=True)
# 기존보다 행 개수가 준 것을 확인할 수 있음 
data_weather_menu.shape
# 가지고 있는 칼럼 보기 -> 더미변수화를 위해

C:\Users\hop09\AppData\Local\Temp\ipykernel_9952\635812491.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_weather_menu.drop(drop_index, inplace=True)


(15307, 11)

In [120]:
# 메뉴, 계절, 요일은 더미변수화해서 보는 것이 좋을 것 같다고 판단, 더미변수화 진행
data_dummies_menu = pd.DataFrame(pd.get_dummies(data_weather_menu['menu']))
data_dummies_season = pd.DataFrame(pd.get_dummies(data_weather_menu['season']))
data_dummies_day = pd.DataFrame(pd.get_dummies(data_weather_menu['day']))
# 새로운 데이터프레임 만들어주기
data_target = data_weather_menu[['labeling']]
data_weather_menu = pd.concat([data_weather_menu[['cloudy',
       'rain', 'snow', 'sunny', 'airpolution', 'humid(%)', 'temperature',
      ]] ,data_dummies_season, data_dummies_day, data_dummies_menu, data_target], axis = 1)
print(data_weather_menu.shape)

(15307, 175)


In [121]:
# 상관분석 진행
pd.set_option('display.max_rows', None)
data_weather_corr = pd.DataFrame(data_weather_menu.corr(method='pearson'))
data_weather_corr
data_weather_corr.to_excel("날씨품명cor분석_코로나전.xlsx", encoding='cp949')

c:\Users\hop09\Desktop\anaconda\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


## 코로나 후 분석

In [122]:
# 데이터 불러오기
import pandas as pd
import numpy as np 
import os

data_origin_bc = pd.read_csv("데이터전처리_혜원_코로나후.CSV", header = 0)


In [123]:
# 날씨, 요일, 메뉴를 보려고 함
import seaborn as sns
import matplotlib.pyplot as plt

data_weather_menu_bc = data_origin_bc[['day', 'season', 'temperature', 'cloudy',
       'rain', 'snow', 'sunny', 'airpolution', 'humid(%)',
       'labeling' , 'menu']]

In [124]:
print(data_weather_menu_bc.shape)
# 상위 5개 품목은 어떤 요인과 관련없이 다 잘나가기 때문에 해당 음료는 제외하고 보기
# 해당 음료가 있는 행의 인덱스 반환 후 인덱스 기준으로 삭제하기
drop_index = data_weather_menu_bc[(data_weather_menu_bc["menu"] == '아이스 아메리카노')|(data_weather_menu_bc["menu"] == '아이스 카페라떼')|(data_weather_menu_bc["menu"] == '아이스 바닐라라떼')|(data_weather_menu_bc["menu"] == '딸기라떼')|(data_weather_menu_bc["menu"] == '아메리카노')].index
#아이스 아메리카노, 아이스 카페라떼, 아이스 바닐라라떼, 딸기라떼, 아메리카노

print(drop_index)
# inplace = True : 변경을 저장하겠다. 

(4901, 11)
Int64Index([   1,    2,    4,    7,   15,   16,   18,   20,   25,   30,
            ...
            4855, 4856, 4857, 4859, 4868, 4880, 4882, 4883, 4886, 4893],
           dtype='int64', length=1173)


In [125]:
data_weather_menu_bc.drop(drop_index, inplace=True)
# 기존보다 행 개수가 준 것을 확인할 수 있음 
data_weather_menu_bc.shape
# 가지고 있는 칼럼 보기 -> 더미변수화를 위해

C:\Users\hop09\AppData\Local\Temp\ipykernel_9952\635812491.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_weather_menu.drop(drop_index, inplace=True)


(3728, 11)

In [126]:
# 메뉴, 계절, 요일은 더미변수화해서 보는 것이 좋을 것 같다고 판단, 더미변수화 진행
data_dummies_menu = pd.DataFrame(pd.get_dummies(data_weather_menu_bc['menu']))
data_dummies_season = pd.DataFrame(pd.get_dummies(data_weather_menu_bc['season']))
data_dummies_day = pd.DataFrame(pd.get_dummies(data_weather_menu_bc['day']))
# 새로운 데이터프레임 만들어주기
data_target = data_weather_menu_bc[['labeling']]
data_weather_menu_bc = pd.concat([data_weather_menu_bc[['cloudy',
       'rain', 'snow', 'sunny', 'airpolution', 'humid(%)', 'temperature',
      ]] ,data_dummies_season, data_dummies_day, data_dummies_menu, data_target], axis = 1)
print(data_weather_menu_bc.shape)

(3728, 90)


In [127]:
# 상관분석 진행
pd.set_option('display.max_rows', None)
data_weather_corr = pd.DataFrame(data_weather_menu.corr(method='pearson'))
data_weather_corr
data_weather_corr.to_excel("날씨품명cor분석_코로나후.xlsx", encoding='cp949')

c:\Users\hop09\Desktop\anaconda\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
